In [6]:
from preprocess import stopwords

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [7]:
import os

pre_file_path = os.path.join('pre_normalized', 'news.csv')
pre_df = pd.read_csv(pre_file_path)
pre_df = pre_df[["text","label"]]
pre_df.head()

,text,label
0,autor bestseller cita 5 expressoes indicam int...,fake
1,juiz df confirma indicios expresidente petista...,fake
2,senhora 60 anos corre atras eduardo cunha aero...,fake
3,russia ameaca derrubar avioes americanos siria...,fake
4,governo podera demitir 18 mil servidores banco...,fake


In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

# bag_of_words_normalized = TfidfVectorizer(strip_accents="ascii", lowercase=True, stop_words = stopwords, use_idf = False, norm='l2')
# bag_of_words_idf = TfidfVectorizer(strip_accents="ascii", lowercase=True, stop_words = stopwords)
#bag_of_words = TfidfVectorizer(strip_accents="ascii", lowercase=True, stop_words = stopwords, use_idf = False, norm='l2')
bag_of_words = TfidfVectorizer(strip_accents="ascii", lowercase=True, stop_words = stopwords)

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(pre_df["text"], pre_df["label"], shuffle= True, test_size=0.2)

bag_of_words.fit(X_train)

X_train = bag_of_words.transform(X_train).toarray()
X_test = bag_of_words.transform(X_test).toarray()

In [10]:
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.svm import NuSVC

svc = SVC()
lsvc = LinearSVC()
nsvc = NuSVC()


support_vector = [svc, lsvc, nsvc]
support_vector_names = ["SVC", "LSVC", "NSVC"]

In [11]:
from collections import defaultdict
scores = defaultdict(list)

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

for algo in support_vector:
    algo.fit(X_train, y_train)
    y_predict = algo.predict(X_test)

    scores["accuracy"].append(accuracy_score(y_test, y_predict))
    scores["precision"].append(precision_score(y_test, y_predict, pos_label = 'fake'))
    scores["recall"].append(recall_score(y_test, y_predict, pos_label = 'fake'))
    scores["f1_score"].append(f1_score(y_test, y_predict, pos_label = 'fake'))

support_vector_df = pd.DataFrame(scores, index = support_vector_names)
support_vector_df

,accuracy,precision,recall,f1_score
SVC,0.905556,0.906944,0.904432,0.905687
LSVC,0.902778,0.906425,0.898892,0.902643
NSVC,0.904861,0.907950,0.901662,0.904795


In [12]:
# from sklearn.model_selection import cross_val_score

# for i, algo in enumerate(support_vector):
#     acc = cross_val_score(algo, X_train, y_train, cv=5, scoring='accuracy')
#     print(f"{algo} mean accuracy: ", end = "")
#     print(round(np.mean(acc), 3))

GaussianNB() mean accuracy: 0.732
BernoulliNB() mean accuracy: 0.876
MultinomialNB() mean accuracy: 0.867

In [20]:
support_vector_df.index.name = "Model"
support_vector_df.to_csv("results/support_vector.csv",index=True)